In [1]:
!pip install transformers==4.31.0 onnx==1.14.0 onnxruntime==1.15.1 optimum==1.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1

In [8]:
from transformers import AutoModelForSequenceClassification
import torch
import onnx

# 1. Download the model
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 2. Convert the model to ONNX
# Adjust input shape to have only batch size and sequence length
dummy_input = torch.randint(0, 30522, (1, 16)).to(torch.int64) # Assume vocab size of 30522, adjust as needed
# Use input_ids and attention_mask as inputs for the export
torch.onnx.export(model, (dummy_input, torch.ones_like(dummy_input)), "bert-base-uncased.onnx",
                  input_names=['input_ids', 'attention_mask']) # Specify input names



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# 3. Load the ONNX model and set up inference
import onnxruntime as ort

# Create an inference session
session = ort.InferenceSession("bert-base-uncased.onnx")

# # Prepare input data
# input_ids = torch.tensor([[101, 2057, 2049, 102]]).to(torch.int64)
# attention_mask = torch.tensor([[1, 1, 1, 1]]).to(torch.int64)

# Prepare input data
# Adjust the input sequence length to match the expected length (16)
input_ids = torch.tensor([[101, 2057, 2049, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]).to(torch.int64)
attention_mask = torch.tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]).to(torch.int64)


# Run the model
input_names = [input.name for input in session.get_inputs()]
output_names = [output.name for output in session.get_outputs()]

outputs = session.run(output_names, {input_names[0]: input_ids.numpy(), input_names[1]: attention_mask.numpy()})

print(outputs)

[array([[-0.02504878, -0.02753285]], dtype=float32)]


In [ ]:
# #### Pre Installs
# import os
# os.environ['openAIAPIKey'] = ''
# os.environ['serpAPIKey'] = ''
!pip install google-search-results

In [ ]:
appConfig = {
    "NoOfLinks" : 3,
    "oaSystemprompt" : "You are an assistant that analyzes the contents of a website and extracts requested details in a json format with provided keys and ignoring and other text. Respond in markdown.",
    "oaUserprompt" : ""
}

In [ ]:
import os

class getApiKeys:

  def __init__(self):
    pass

  def getApiKeys(self,apiKey):
    return os.environ.get(apiKey)

In [ ]:
import serpapi
import json
from serpapi import GoogleSearch # Add this import statement

class searchForLinks:
  def __init__(self):
    pass

  def getSearchResults(self,searchWord):
    searpApiKey = getApiKeys()
    params = {
      "q": searchWord,
      "hl": "en",
      "gl": "us",
      "google_domain": "google.com",
      "api_key": ''
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    linksList = []

    if 'organic_results' in results:
        for res in results['organic_results']:
            if 'link' in res:  # Check if 'link' key exists in each result dictionary
                linksList.append(res['link'])
    else:
        print("No organic results found in the response.")

    return linksList[:appConfig['NoOfLinks']]

In [ ]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display


class webCrawler:

  def __init__(self):
    pass

  def crawlresults(self,srch):
    sfl = searchForLinks()
    linktocrawl = sfl.getSearchResults(srch)

    crawlJson = {}
    for links in linktocrawl:
      crawlJson.update(self.crawl(links))

    return crawlJson

  def crawl(self, url: str):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.title.string if soup.title else "Title not found"

    if soup.body:
      for irrelevant in soup.body(["script", "style","img","input"]):
        irrelevant.decompose()
      text = soup.body.get_text(separator="\n", strip=True)
    else:
      text = ""

    return {title : text}

    # links = [link.get('href') for link in soup.find_all('a')]
    # self.links = [link for link in links if link]

In [ ]:
import os
import requests
# from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import time


class summarizer:

  def __init__(self):
    pass

  def user_prompt_for(self,title,text):
      user_prompt = f"You are looking at a website titled {title}"
      user_prompt += "The contents of this website is as follows; please find any value about APY, Terms rewards and respond as as JSON with following format {'apy':'','fees':''}. ignore anything else.\n\n"
      user_prompt += text
      return user_prompt

  def messages_for(self,content):
      return [
          {"role": "system", "content": appConfig['oaSystemprompt']},
          {"role": "user", "content": content}
      ]

  def summarize(self,url):
      searpApiKey = getApiKeys()
      api_key = searpApiKey.getApiKeys('openAIAPIKey')

      openai = OpenAI(api_key=api_key)

      crawl = webCrawler()
      contents = crawl.crawlresults(url)

      responses = []

      for content in contents:
        user_prompt = self.user_prompt_for(content,contents[content])
        messages = self.messages_for(content)

        response = openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = messages
        )
        responses.append(response.choices[0].message.content)

      return responses


In [ ]:
extract = summarizer()
results = extract.summarize('chase freedom credit card')
for I in results:
  display(Markdown(I))

```json
{
  "title": "Chase Freedom",
  "category": "Credit Cards",
  "website": "Chase.com"
}
```

```json
{
  "card_name": "Chase Freedom Unlimited",
  "issuer": "Chase",
  "rewards": {
    "cash_back": "1.5% on all purchases",
    "bonus_offer": "Earn a $200 bonus after you spend $500 on purchases in the first 3 months from account opening",
    "categories": {
      "dining": "3% on dining at restaurants, including takeout and eligible delivery services",
      "drugs": "3% at drugstores",
      "travel": "5% on travel purchased through Chase Ultimate Rewards"
    }
  },
  "annual_fee": "$0",
  "foreign_transaction_fee": "$0",
  "intro_offer": "0% intro APR on purchases for 15 months, then a variable APR of 19.24% - 27.49%",
  "credit_score_needed": "Good to Excellent",
  "other_benefits": [
    "Purchase protection",
    "Extended warranty",
    "No foreign transaction fees"
  ]
}
```

```json
{
  "title": "Which Chase Freedom Credit Card Should You Get?",
  "author": "NerdWallet",
  "date": "2023-10",
  "summary": "A guide to choosing the best Chase Freedom credit card based on individual spending habits and rewards preferences.",
  "key_points": [
    "Overview of Chase Freedom credit card options.",
    "Comparison of benefits and rewards among different Chase Freedom cards.",
    "Tips for maximizing rewards based on spending categories.",
    "Considerations for new applicants and existing Chase customers."
  ]
}
```